In [1]:
import os
import re  
import torch
from torch_geometric.utils import to_dense_adj  
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import gc 
def orden_numerico(archivo):
    numero = re.search(r'\d+', archivo)
    if numero:
        return int(numero.group())
    return 0

ruta_guardado = "Dades/Snapshots"

if not os.path.exists(ruta_guardado):
    raise RuntimeError(f"La ruta especificada no existe: {ruta_guardado}")

snapshots_reestructurados = []
num_archivos_intentados = 0
num_archivos_cargados = 0

archivos = [archivo for archivo in os.listdir(ruta_guardado) if archivo.endswith('.pt')]

archivos_ordenados = sorted(archivos, key=orden_numerico)

for archivo in archivos_ordenados:
    num_archivos_intentados += 1
    ruta_archivo = os.path.join(ruta_guardado, archivo)
    try:
        snapshot = torch.load(ruta_archivo)
        print(f"cargando el archivo {archivo}")
        snapshots_reestructurados.append(snapshot)
        del snapshot
        gc.collect()
        
        
        num_archivos_cargados += 1
    except Exception as e:
        print(f"Error al cargar el archivo {archivo}: {e}")

print(f"{num_archivos_cargados} de {num_archivos_intentados} archivos han sido cargados correctamente.")

snapshots_reestructurados[0]

cargando el archivo snapshot_6.pt
cargando el archivo snapshot_7.pt
cargando el archivo snapshot_8.pt
cargando el archivo snapshot_9.pt
cargando el archivo snapshot_10.pt
cargando el archivo snapshot_11.pt
cargando el archivo snapshot_12.pt
cargando el archivo snapshot_13.pt
cargando el archivo snapshot_14.pt
cargando el archivo snapshot_15.pt
cargando el archivo snapshot_16.pt
cargando el archivo snapshot_17.pt
cargando el archivo snapshot_18.pt
cargando el archivo snapshot_19.pt
cargando el archivo snapshot_20.pt
cargando el archivo snapshot_21.pt
cargando el archivo snapshot_22.pt
cargando el archivo snapshot_23.pt
cargando el archivo snapshot_24.pt
cargando el archivo snapshot_25.pt
cargando el archivo snapshot_26.pt
cargando el archivo snapshot_27.pt
cargando el archivo snapshot_28.pt
cargando el archivo snapshot_29.pt
cargando el archivo snapshot_30.pt
cargando el archivo snapshot_31.pt
cargando el archivo snapshot_32.pt
cargando el archivo snapshot_33.pt
cargando el archivo snap

Data(x=[512, 441], edge_index=[2, 8310], edge_attr=[8310, 1], y=[512, 48], coordinates=[512, 2])

In [2]:
snapshots_reestructurados[0]

Data(x=[512, 441], edge_index=[2, 8310], edge_attr=[8310, 1], y=[512, 48], coordinates=[512, 2])

In [3]:
import torch
from torch_geometric.data import Data
     
from torch_geometric.loader import DataLoader
total_datos = len(snapshots_reestructurados)
train_size = int(0.7 * total_datos)
val_size = int(0.15 * total_datos)
test_size = total_datos - train_size - val_size

train_dataset = snapshots_reestructurados[:train_size]
val_dataset = snapshots_reestructurados[train_size:train_size + val_size]
test_dataset = snapshots_reestructurados[train_size + val_size:]

caracteristicas_concatenadas = torch.cat([data.x for data in train_dataset], dim=0).to(torch.float64)
mean = caracteristicas_concatenadas.mean(dim=0)
std = caracteristicas_concatenadas.std(dim=0)

umbral_std = 1e-6
std[std < umbral_std] = 1

def normalize_dataset(dataset):
    for data in dataset:
        data.x = ((data.x.to(torch.float64) - mean) / std).to(torch.float32)  
    return dataset

train_dataset = normalize_dataset(train_dataset)
val_dataset = normalize_dataset(val_dataset)
test_dataset = normalize_dataset(test_dataset)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))


batch_size = 1 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_loader:
    print("Tamaño del lote (número de grafos):", batch.num_graphs)
    print("Características del primer grafo del lote:", batch[0].x.shape)
    print("Etiquetas del primer grafo del lote:", batch[0].y.shape)
    break
del total_datos, snapshots_reestructurados

18404
3943
3945
Tamaño del lote (número de grafos): 1
Características del primer grafo del lote: torch.Size([512, 441])
Etiquetas del primer grafo del lote: torch.Size([512, 48])


In [4]:
"""import torch
from torch import nn

class Conv2DLSTMModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(Conv2DLSTMModel, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=1, padding=(1, 1))
        self.lstm = nn.LSTM(input_size=32, hidden_size=64, batch_first=True) 
        self.linear = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(-1, 1, 1, 270)
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1, 32)  
        x, (hn, cn) = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(x)
        return x
      


NUM_FEATURES = 270
NUM_CLASSES = 48

model = Conv2DLSTMModel(num_features=NUM_FEATURES, num_classes=NUM_CLASSES)"""

'import torch\nfrom torch import nn\n\nclass Conv2DLSTMModel(nn.Module):\n    def __init__(self, num_features, num_classes):\n        super(Conv2DLSTMModel, self).__init__()\n        self.conv = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=1, padding=(1, 1))\n        self.lstm = nn.LSTM(input_size=32, hidden_size=64, batch_first=True) \n        self.linear = nn.Linear(64, num_classes)\n\n    def forward(self, x):\n        x = x.view(-1, 1, 1, 270)\n        x = self.conv(x)\n        x = torch.relu(x)\n        x = x.view(x.size(0), -1, 32)  \n        x, (hn, cn) = self.lstm(x)\n        x = x[:, -1, :]\n        x = self.linear(x)\n        return x\n      \n\n\nNUM_FEATURES = 270\nNUM_CLASSES = 48\n\nmodel = Conv2DLSTMModel(num_features=NUM_FEATURES, num_classes=NUM_CLASSES)'

In [5]:
"""import torch.nn as nn

class Conv3DLSTMModel(nn.Module):
    def __init__(self, num_stations, num_time_steps, num_features_per_time_step, num_classes, dropout_rate=0.1):
        super(Conv3DLSTMModel, self).__init__()
        self.num_stations = num_stations
        self.num_time_steps = num_time_steps
        self.num_features_per_time_step = num_features_per_time_step
        self.num_classes = num_classes
        
        self.conv3d = nn.Conv3d(in_channels=1, out_channels=num_features_per_time_step, kernel_size=(3, 3, 3), stride=1, padding=(1, 1, 1))
        self.bn = nn.BatchNorm3d(num_features_per_time_step)
        self.relu = nn.ReLU(inplace=True)
        self.dropout_conv = nn.Dropout3d(p=dropout_rate) 
        lstm_input_size = num_features_per_time_step * num_features_per_time_step
        self.hidden_size = 256 
        hidden_dim = 8  
        
        self.lstm = nn.LSTM(input_size=lstm_input_size, hidden_size=hidden_dim, batch_first=False)
        self.dropout_lstm = nn.Dropout(p=dropout_rate) 
        
        self.linear = nn.Linear(hidden_dim * num_time_steps, self.num_classes) 

    def forward(self, x):
        x = x.view(-1, 1, self.num_time_steps, self.num_stations, self.num_features_per_time_step)
        x = self.conv3d(x)
        #x = self.bn(x)
        x = self.relu(x)
        x = self.dropout_conv(x) 
        x = x.permute(2, 3, 0, 1, 4)
        x = x.reshape(self.num_time_steps, self.num_stations, -1)  
        x, (hn, cn) = self.lstm(x)
        x = self.dropout_lstm(x)  
        x = x.permute(1, 0, 2)
        x = x.reshape(self.num_stations, -1)
        x = self.linear(x)
        return x


NUM_STATIONS = 512 
NUM_TIME_STEPS = 7  
NUM_FEATURES_PER_TIME_STEP = 63  
NUM_CLASSES = 48 

model = Conv3DLSTMModel(num_stations=NUM_STATIONS, num_time_steps=NUM_TIME_STEPS, num_features_per_time_step=NUM_FEATURES_PER_TIME_STEP, num_classes=NUM_CLASSES)"""


'import torch.nn as nn\n\nclass Conv3DLSTMModel(nn.Module):\n    def __init__(self, num_stations, num_time_steps, num_features_per_time_step, num_classes, dropout_rate=0.1):\n        super(Conv3DLSTMModel, self).__init__()\n        self.num_stations = num_stations\n        self.num_time_steps = num_time_steps\n        self.num_features_per_time_step = num_features_per_time_step\n        self.num_classes = num_classes\n        \n        self.conv3d = nn.Conv3d(in_channels=1, out_channels=num_features_per_time_step, kernel_size=(3, 3, 3), stride=1, padding=(1, 1, 1))\n        self.bn = nn.BatchNorm3d(num_features_per_time_step)\n        self.relu = nn.ReLU(inplace=True)\n        self.dropout_conv = nn.Dropout3d(p=dropout_rate) \n        lstm_input_size = num_features_per_time_step * num_features_per_time_step\n        self.hidden_size = 256 \n        hidden_dim = 8  \n        \n        self.lstm = nn.LSTM(input_size=lstm_input_size, hidden_size=hidden_dim, batch_first=False)\n        sel

In [12]:
import torch
from torch import nn

class Conv3DLSTMModel(nn.Module):
    def __init__(self, num_stations, num_time_steps, num_features_per_time_step, num_classes):
        super(Conv3DLSTMModel, self).__init__()
        self.conv = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(3, 3, 3), stride=1, padding=(1, 1, 1))
        self.lstm = nn.LSTM(input_size=32 * num_features_per_time_step, hidden_size=64, batch_first=True)
        self.linear = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(-1, NUM_TIME_STEPS, NUM_FEATURES_PER_TIME_STEP).unsqueeze(1)  # [1521, 1, 7, 25]
        x = x.unsqueeze(1)  
        
        x = self.conv(x) 
        x = torch.relu(x)
        x = x.view(512, -1, 32 * 63)  

        x, (hn, cn) = self.lstm(x)
        x = x[:, -1, :]  
        
        x = self.linear(x)  # Salida: [1521, 42]

        return x

NUM_STATIONS = 512 
NUM_TIME_STEPS = 7  
NUM_FEATURES_PER_TIME_STEP = 63  
NUM_CLASSES = 48 

model = Conv3DLSTMModel(NUM_STATIONS, NUM_TIME_STEPS, NUM_FEATURES_PER_TIME_STEP, NUM_CLASSES)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.HuberLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10, verbose=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

def train(model, train_loader):
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        
        optimizer.zero_grad()
        out = model(data.x)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate(model, val_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            
            out = model(data.x)
            loss = criterion(out, data.y)
            total_loss += loss.item()
    return total_loss / len(val_loader)

num_epochs = 100
best_val_loss = float('inf')
best_model = None

for epoch in range(num_epochs):
    train_loss = train(model, train_loader)
    val_loss = validate(model, val_loader)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
    scheduler.step(val_loss)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict()
        torch.save(model.state_dict(), 'best_model.pth')

model.load_state_dict(torch.load('best_model.pth'))


c:\Users\marcs\miniconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch: 1, Train Loss: 3.0594, Validation Loss: 3.3864
Epoch: 2, Train Loss: 2.9446, Validation Loss: 3.3617
Epoch: 3, Train Loss: 2.8926, Validation Loss: 3.3392
Epoch: 4, Train Loss: 2.8596, Validation Loss: 3.3263
Epoch: 5, Train Loss: 2.8368, Validation Loss: 3.3232
Epoch: 6, Train Loss: 2.8201, Validation Loss: 3.3157
Epoch: 7, Train Loss: 2.8068, Validation Loss: 3.3210
Epoch: 8, Train Loss: 2.7962, Validation Loss: 3.3299
Epoch: 9, Train Loss: 2.7881, Validation Loss: 3.3414
Epoch: 10, Train Loss: 2.7803, Validation Loss: 3.3351
Epoch: 11, Train Loss: 2.7729, Validation Loss: 3.3247
Epoch: 12, Train Loss: 2.7668, Validation Loss: 3.3278
Epoch: 13, Train Loss: 2.7615, Validation Loss: 3.3187
Epoch: 14, Train Loss: 2.7579, Validation Loss: 3.3125
Epoch: 15, Train Loss: 2.7542, Validation Loss: 3.3055
Epoch: 16, Train Loss: 2.7508, Validation Loss: 3.3051
Epoch: 17, Train Loss: 2.7475, Validation Loss: 3.3081
Epoch: 18, Train Loss: 2.7446, Validation Loss: 3.3028
Epoch: 19, Train Lo

<All keys matched successfully>

In [18]:
import numpy as np
import torch
import matplotlib.pyplot as plt

#model.load_state_dict(torch.load('best_model.pth'))
nombres_compuestos = ["C6H6", "CO", "NO2", "NOx", "O3", "PM10", "PM25", "SO2"] 

 
def calculate_metrics(outputs, targets):
    mse = criterion(outputs, targets)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(outputs - targets))
    mape = torch.mean(torch.abs((targets - outputs) / torch.clamp(targets, min=1e-8))) * 100

    outputs_np = outputs.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()

    ss_res = np.sum((targets_np - outputs_np) ** 2)
    ss_tot = np.sum((targets_np - np.mean(targets_np)) ** 2)
    r_squared = 1 - (ss_res / ss_tot) if ss_tot != 0 else np.nan

    if np.std(outputs_np) > 0 and np.std(targets_np) > 0:
        correlation = np.corrcoef(outputs_np.ravel(), targets_np.ravel())[0, 1]
    else:
        correlation = np.nan

    tolerance = 0.1 * np.mean(targets_np)
    accuracy = np.mean(np.abs(outputs_np - targets_np) < tolerance)

    return mse.item(), rmse.item(), mae.item(), mape.item(), r_squared, correlation, accuracy

def test(model, test_loader, num_compounds, num_steps, nombres_compuestos):
    model.eval()
    device = next(model.parameters()).device
    metrics_per_step_compound = [[[] for _ in range(num_steps)] for _ in range(num_compounds)]

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data.x)
            out = out.view(-1, num_compounds, num_steps).to(device)
            y = data.y.view(-1, num_compounds, num_steps).to(device)

            for j in range(num_compounds):
                for i in range(num_steps):
                    metrics = calculate_metrics(out[:, j, i], y[:, j, i])
                    metrics_per_step_compound[j][i].append(metrics)

    averaged_metrics = [[np.mean(metrics, axis=0) for metrics in step_compounds] for step_compounds in metrics_per_step_compound]
    return averaged_metrics

num_compounds = 8 
num_steps = 6     
averaged_metrics = test(model, test_loader, num_compounds, num_steps, nombres_compuestos)

medias_por_compuesto = []
for j in range(num_compounds):
    metrics_compound = [averaged_metrics[j][i] for i in range(num_steps)]
    metrics_array = np.array(metrics_compound)
    mean_metrics = np.mean(metrics_array, axis=0)
    medias_por_compuesto.append(mean_metrics)

for i, mean_metrics in enumerate(medias_por_compuesto):
    mse, rmse, mae, mape, r_squared, corr, accuracy = mean_metrics
    print(f"Media para {nombres_compuestos[i]} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}, R^2: {r_squared:.4f}, Correlación: {corr:.4f}, Precisión: {accuracy:.4f}") 

Media para C6H6 - MSE: 0.0375, RMSE: 0.1879, MAE: 0.1605, MAPE: 129959999.6512, R^2: 0.5122, Correlación: 0.7416, Precisión: 0.2934
Media para CO - MSE: 0.0031, RMSE: 0.0532, MAE: 0.0467, MAPE: 1216010.5264, R^2: 0.7134, Correlación: 0.8577, Precisión: 0.4672
Media para NO2 - MSE: 3.8171, RMSE: 1.8996, MAE: 4.2465, MAPE: 72698724.9557, R^2: 0.5977, Correlación: 0.8160, Precisión: 0.3917
Media para NOx - MSE: 6.2444, RMSE: 2.4335, MAE: 6.6983, MAPE: 113934463.0070, R^2: 0.5979, Correlación: 0.8150, Precisión: 0.3973
Media para O3 - MSE: 7.2742, RMSE: 2.6586, MAE: 7.7487, MAPE: 842774034.1844, R^2: 0.6805, Correlación: 0.8550, Precisión: 0.5126
Media para PM10 - MSE: 3.2246, RMSE: 1.7639, MAE: 3.6442, MAPE: 86113437.2855, R^2: 0.6026, Correlación: 0.7893, Precisión: 0.5302
Media para PM25 - MSE: 1.2460, RMSE: 1.0991, MAE: 1.6136, MAPE: 108424557.7651, R^2: 0.5910, Correlación: 0.7820, Precisión: 0.5399
Media para SO2 - MSE: 0.3664, RMSE: 0.5908, MAE: 0.5834, MAPE: 47132024.2734, R^2: 0.5

In [ ]:
import numpy as np
import torch
nombres_compuestos = ["C6H6", "CO", "NO2", "NOx", "O3", "PM10", "PM25", "SO2"]
def calculate_metrics(outputs, targets):
    mse = criterion(outputs, targets)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(outputs - targets))
 
    outputs_np = outputs.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()

    if np.std(outputs_np) > 0 and np.std(targets_np) > 0:
        correlation = np.corrcoef(outputs_np.ravel(), targets_np.ravel())[0, 1]
    else:
        correlation = np.nan

    return mse.item(), rmse.item(), mae.item(), correlation

def test(model, test_loader, num_compounds, num_steps, nombres_compuestos):
    model.eval()
    device = next(model.parameters()).device  
    metrics_per_step_compound = [[[[] for _ in range(num_steps)] for _ in range(num_compounds)]]

    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            
            out = model(data.x)
          
            out = out.view(-1, num_compounds, num_steps ).to(device)
            y = data.y.view(-1, num_compounds, num_steps).to(device)
            
            for j in range(num_compounds):
                for i in range(num_steps):
                    mse, rmse, mae, corr = calculate_metrics(out[:, j, i], y[:, j, i])
                    metrics_per_step_compound[0][j][i].append((mse, rmse, mae, corr))

    averaged_metrics = [[[np.mean(metrics, axis=0) for metrics in step_compounds] for step_compounds in step_metrics] for step_metrics in metrics_per_step_compound]

    return averaged_metrics


num_compounds = 8 
num_steps = 6     
averaged_metrics = test(model, test_loader, num_compounds, num_steps, nombres_compuestos)


In [ ]:
for j in range(num_compounds):
        for i in range(num_steps):
            mse, rmse, mae, corr = averaged_metrics[0][j][i]
            print(f"Paso {i+1}, Compuesto {nombres_compuestos[j]} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

Paso 1, Compuesto C6H6 - MSE: 0.0859, RMSE: 0.2859, MAE: 0.2962
Paso 2, Compuesto C6H6 - MSE: 0.0860, RMSE: 0.2859, MAE: 0.2962
Paso 3, Compuesto C6H6 - MSE: 0.0867, RMSE: 0.2868, MAE: 0.2979
Paso 4, Compuesto C6H6 - MSE: 0.0876, RMSE: 0.2880, MAE: 0.2999
Paso 5, Compuesto C6H6 - MSE: 0.0883, RMSE: 0.2891, MAE: 0.3017
Paso 6, Compuesto C6H6 - MSE: 0.0891, RMSE: 0.2903, MAE: 0.3034
Paso 1, Compuesto CO - MSE: 0.0100, RMSE: 0.0984, MAE: 0.0899
Paso 2, Compuesto CO - MSE: 0.0100, RMSE: 0.0985, MAE: 0.0899
Paso 3, Compuesto CO - MSE: 0.0100, RMSE: 0.0988, MAE: 0.0901
Paso 4, Compuesto CO - MSE: 0.0101, RMSE: 0.0990, MAE: 0.0903
Paso 5, Compuesto CO - MSE: 0.0101, RMSE: 0.0991, MAE: 0.0903
Paso 6, Compuesto CO - MSE: 0.0101, RMSE: 0.0991, MAE: 0.0903
Paso 1, Compuesto NO2 - MSE: 7.0524, RMSE: 2.5896, MAE: 7.5256
Paso 2, Compuesto NO2 - MSE: 7.0549, RMSE: 2.5882, MAE: 7.5277
Paso 3, Compuesto NO2 - MSE: 7.0771, RMSE: 2.5903, MAE: 7.5497
Paso 4, Compuesto NO2 - MSE: 7.0955, RMSE: 2.5929, MAE:

In [ ]:
import numpy as np


medias_por_compuesto = []

for j in range(num_compounds):
    metrics_compound = [averaged_metrics[0][j][i] for i in range(num_steps)]  
    metrics_array = np.array(metrics_compound) 
    mean_metrics = np.mean(metrics_array, axis=0)  
    medias_por_compuesto.append(mean_metrics)

for i, mean_metrics in enumerate(medias_por_compuesto):
    mse, rmse, mae, corr = mean_metrics
    print(f"Media para {nombres_compuestos[i]} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

Media para C6H6 - MSE: 0.0873, RMSE: 0.2876, MAE: 0.2992
Media para CO - MSE: 0.0101, RMSE: 0.0988, MAE: 0.0901
Media para NO2 - MSE: 7.0883, RMSE: 2.5929, MAE: 7.5616
Media para NOx - MSE: 11.7161, RMSE: 3.3342, MAE: 12.1994
Media para O3 - MSE: 16.4257, RMSE: 4.0196, MAE: 16.9179
Media para PM10 - MSE: 6.7049, RMSE: 2.5379, MAE: 7.1754
Media para PM25 - MSE: 2.6490, RMSE: 1.6041, MAE: 3.0886
Media para SO2 - MSE: 1.0201, RMSE: 1.0061, MAE: 1.4066
